<a href="https://colab.research.google.com/github/lingyuhaunti/tensorflow/blob/master/estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
from tensorflow.python.keras.datasets import cifar10
import pydot
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:
def identity_block(X, f, filters,stage, block):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  
  F1, F2, F3 = filters
  
  X_shortcut = X
  
  X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base+'2c')(X)
  
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)
  
  return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
  conv_name_base = 'res' + str(stage) + block +'_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  
  F1, F2, F3 = filters
  
  X_shortcut = X
  
  X = Conv2D(F1, (1, 1), strides = (s, s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(F2, (f, f), strides = (1, 1), name = conv_name_base + '2b', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(F3, (1, 1), strides = (1, 1), name = conv_name_base + '2c', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  
  X_shortcut = Conv2D(F3, (1,1), strides=(s, s), name=conv_name_base+'1', padding='valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name=bn_name_base+'1')(X_shortcut)
  
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)
  
  return X

In [0]:
def ResNet50(input_shape = (32, 32, 3), classes = 10):
  X_input = Input(input_shape)
  
  X = ZeroPadding2D((3, 3))(X_input)
  
  X = Conv2D(64, (7, 7), strides=(2, 2), name = 'conv1', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3, 3), strides=(2,2))(X)
  
  X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block ='a', s=1)
  X = identity_block(X, 3, [64, 64, 256], stage = 2, block = 'b')
  X = identity_block(X, 3, [64, 64, 256], stage = 2, block = 'c')
  
  X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
  X = identity_block(X, f=3, filters=[128, 128, 512], stage = 3, block = 'b')
  X = identity_block(X, f=3, filters=[128, 128, 512], stage = 3, block = 'c')
  X = identity_block(X, f=3, filters=[128, 128, 512], stage = 3, block = 'd')
  
  X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
  X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='b')
  X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='c')
  X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='d')
  X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='e')
  X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='f')

  X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
  X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='b')
  X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='c')
  
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc'+str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
  
  model = Model(inputs = X_input, outputs = X, name='ResNet50')
  
  return model

In [0]:
def input_fn():
  X = np.random.random((1, 32, 32, 3))
  Y = np.random.random((1, 10))
  dataset = tf.data.Dataset.from_tensor_slices((X, Y))
  dataset = dataset.repeat(10)
  dataset = dataset.batch(128)
  return dataset

In [0]:
model = ResNet50(input_shape = (32, 32, 3), classes = 10)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [109]:
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train =  keras.utils.to_categorical(Y_train_orig,10)
Y_test =  keras.utils.to_categorical(Y_test_orig,10)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
#mX_trian = np.random.random((320, 96)).reshape(10,32,32,3)
#mY_trian = np.random.random((32, 96)).reshape(1,32,32,3)
mX_train = np.random.random((100,100))
mY_train = np.random.random((100,100))
print (mX_trian.shape)
print (mY_trian.shape)

number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)
(1000, 32, 32, 3)
(1000, 10)


In [111]:
input_tensor = Input(shape=(224, 224, 3))
model_dir = '/samle_data/'
#keras_model = tf.keras.applications.ResNet50(input_tensor=input_tensor, weights = None)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
#keras_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
keras_estimator = tf.keras.estimator.model_to_estimator(keras_model = model)
#keras_estimator.train(input_fn = input_fn, steps = 10)
#eval_result = keras_estimator.evaluate(input_fn=input_fn)
#print('Eval result: {}'.format(eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8irgp396', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd256c8ef98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={model.input_names[0]: X_train.astype(np.float32)},
  y=Y_train.astype(np.float32),
  num_epochs = 5,
  shuffle=True)

keras_estimator.train(input_fn=train_input_fn, steps=50)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmp8irgp396/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmp8irgp396/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: conv1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: bn_conv1/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: bn_conv1/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: res2a_branch2a/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: res2a_branch2a

In [0]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 128)

In [0]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [30]:
!ls

sample_data


In [63]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys

import numpy as np
import tensorflow as tf


def input_fn():
  x = np.random.random((1024, 10))
  y = np.random.randint(2, size=(1024, 1))
  x = tf.cast(x, tf.float32)
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.repeat(10)
  dataset = dataset.batch(32)
  return dataset


def main(args):
  model_dir = '/sample_data/'
  print('Using %s to store checkpoints.' % model_dir)

  # Define a Keras Model.
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(10,)))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Compile the model.
  optimizer = tf.train.GradientDescentOptimizer(0.2)
  model.compile(loss='binary_crossentropy', optimizer=optimizer)
  tf.keras.backend.set_learning_phase(True)

  # Define a DistributionStrategy and convert the Keras Model to an
  # Estimator that utilizes the DistributionStrategy.

  keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=model, model_dir=model_dir)

  # Train and evaluate the model.
  keras_estimator.train(input_fn=input_fn, steps=10)
  eval_result = keras_estimator.evaluate(input_fn=input_fn)
  print('Eval result: {}'.format(eval_result))


if __name__ == '__main__':
  tf.app.run(argv=sys.argv)

Using /sample_data/ to store checkpoints.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/sample_data/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd26cb86d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done

ValueError: ignored

In [60]:
!ls

-f  sample_data
